In [2]:
# ipeadata module gets Brazilian economic data, obained from https://github.com/ipea/pyIpeaData
# Note: pip install doesn't seem to be working
from pandas import *
from numpy import *
from pprint import pprint
from datetime import date
import time
import ipeadata

Documentação dos metadados: http://ipeadata2-homologa.ipea.gov.br/api/

In [ ]:
meta = ipeadata.get_metadata()

----

Filtros preliminares: excluímos os dados Regionais e selecionamos uma única frequência

In [ ]:
simple_meta = meta[['TEMNOME','SERCODIGO','SERNOME','SERMINDATA','SERMAXDATA','PERNOME','FNTNOME','BASNOME']]
simple_meta = simple_meta[simple_meta.BASNOME=='Macroeconômico']
simple_freq = simple_meta[simple_meta['PERNOME']=='Mensal']
simple_freq = simple_freq.drop(['BASNOME','PERNOME'], axis=1)

Lista de temas:

In [ ]:
simple_freq['TEMNOME'].unique()

Investigamos, por exemplo, quais são os dados relacionados a emprego com informações recentes.

In [ ]:
datasets_emprego = simple_freq[simple_freq['TEMNOME']=='Emprego'].drop('TEMNOME',axis=1)

series = datasets_emprego[['SERCODIGO','SERNOME','FNTNOME','SERMINDATA','SERMAXDATA']].to_dict(orient='index')
series = [series[s] for s in series if '2018' in series[s]['SERMAXDATA']]
pprint(series) 

In [ ]:
df = ipeadata.ipeadata('CNI12_HTRAB12')
df.tail()

Dada a estrutura acima,

In [ ]:
def fixed_series(series):
    df = ipeadata.ipeadata(series)
    time_axis = df.apply(lambda row:datetime(row['ANO'],row['MES'],row['DIA']),axis=1)
    df = DataFrame(df['VALVALOR'])
    df = df.set_index(time_axis)
    df = df.rename(columns={'VALVALOR':series})
    return df

In [ ]:
horas_industria = fixed_series('CNI12_HTRAB12')

In [ ]:
import matplotlib.pyplot as plt
import seaborn

def phase_plot(data):
    plt.scatter(data, data.diff())
    plt.xlabel(data.columns[0])
    plt.ylabel('$\Delta$({})'.format(data.columns[0]))
    plt.show()

In [ ]:
phase_plot(horas_industria)

In [ ]:
horas_industria.plot()

In [ ]:

horas_industria.diff().plot()